# Importing Modules, Getting Web Browser Running, and Importing Ticker Info

In [95]:
# Importing modules
from selenium import webdriver
from selenium.webdriver.edge.service import Service
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import requests
import csv

In [92]:
# Getting the webdriver up and running
# Specifying the path to msedgedriver.exe
driver_path = r"C:\Program Files\msedgedriver.exe"

# Creating a service object
service = Service(executable_path=driver_path)

# Initializing the Edge browser
driver = webdriver.Edge(service=service)

In [93]:
# Logging into Estimize using the Edge browser
estimize_url = 'https://www.estimize.com/users/sign_in'

driver.get(estimize_url)

In [4]:
# Importing the ticker list and URLs to their pages
ticker_list = pd.read_csv('ticker_list.csv')
ticker_list.head()

,ticker,URL
0,AMZN,https://www.estimize.com/AMZN
1,AAPL,https://www.estimize.com/AAPL
2,MSFT,https://www.estimize.com/MSFT
3,GOOG,https://www.estimize.com/GOOG
4,TSLA,https://www.estimize.com/TSLA


# Pulling Basic Ticker Info from the URLs

In [5]:
# Making an empty list in which to store the scraped data
ticker_data = []


# Looping through the ticker URLs and pulling the general ticker info
# Setting the ticker_code local variable as the already known ticker code and setting the ticker_url to iterate through
for index, row in ticker_list.iterrows():
    ticker_code = row['ticker']
    ticker_url = row['URL']

    # Navigating to each ticker URL
    driver.get(ticker_url)

    # Pulling the needed basic ticker information
    ticker_name = driver.find_element(By.CSS_SELECTOR, '#releases_show > div.main-content-wrapper.is-closed > div:nth-child(2) > div > div:nth-child(1) > div > div > div > div.release-header-information-title-description > p > a').text
    sector = driver.find_element(By.CSS_SELECTOR, '#releases_show > div.main-content-wrapper.is-closed > div:nth-child(2) > div > div:nth-child(1) > div > div > div > p > span:nth-child(1) > a > span').text
    industry = driver.find_element(By.CSS_SELECTOR, '#releases_show > div.main-content-wrapper.is-closed > div:nth-child(2) > div > div:nth-child(1) > div > div > div > p > span:nth-child(2) > a > span').text
    followers = driver.find_element(By.CSS_SELECTOR, '#summary-stats > div > div > div:nth-child(1) > div.estimize-row.summary-sub-title').text
    analysts = driver.find_element(By.CSS_SELECTOR, '#summary-stats > div > div > div:nth-child(2) > a').text

    # Placing the pulled data into the ticker_data list as a dictionary
    ticker_data.append({
        'Ticker': ticker_code,
        'Name': ticker_name,
        'Sectors': sector,
        'Industries': industry,
        'Followers': followers,
        'Analysts': analysts
    })

# Making a DataFrame out of ticker_data
ticker_df = pd.DataFrame(ticker_data)

# Printing the new DataFrame
print(ticker_df)

   Ticker                          Name                 Sectors  \
0    AMZN               Amazon.com Inc.  Consumer Discretionary   
1    AAPL                    Apple Inc.  Information Technology   
2    MSFT         Microsoft Corporation  Information Technology   
3    GOOG                 Alphabet Inc.  Information Technology   
4    TSLA                    Tesla Inc.  Consumer Discretionary   
5     JNJ             Johnson & Johnson             Health Care   
6      PG          Procter & Gamble Co.        Consumer Staples   
7    NVDA            NVIDIA Corporation  Information Technology   
8    CSCO           Cisco Systems, Inc.  Information Technology   
9    BABA    Alibaba Group Holding Ltd.  Consumer Discretionary   
10     HD          The Home Depot, Inc.  Consumer Discretionary   
11   BIDU                   Baidu, Inc.  Information Technology   
12    WMT          Wal-Mart Stores Inc.        Consumer Staples   
13    CRM                Salesforce.com  Information Technolog

In [6]:
# Saving ticker_df as a csv file
ticker_df.to_csv('ticker_data.csv', index=False)

# Pulling the EPS Data

In [16]:
#Preparing an empty list for EPS data storage
eps_dfs = []

# Pulling the EPS data for each ticker
for index, row in ticker_list.iterrows():
    ticker = row['ticker']
    table_url = f"https://www.estimize.com/{ticker}/fq2-2021?metric_name=eps&chart=table"

    # Navigating to the table URL
    driver.get(table_url)

    time.sleep(3)
    # Selecting the table and storing the data in local variable
    table = driver.find_element(By.CSS_SELECTOR, "#main-content > div:nth-child(2) > div:nth-child(3) > div:nth-child(4) > div > table")
    
    # Putting the table data into a DataFrame
    eps_df = pd.read_html(table.get_attribute('outerHTML'))[0]

    # Removing the first row of the data (user data which is all blank)
    eps_df = eps_df.iloc[1:, :]

    # Reseting the row indeces
    eps_df.reset_index(drop=True, inplace=True)

    # Adding a Ticker columnn to differentiate the tables before combining the dataframe
    eps_df['Ticker'] = ticker
    
    # Putting the table data into the eps_dfs list
    eps_dfs.append(eps_df)

# Concatenating all the DataFrames into a single DataFrame
combined_eps_df = pd.concat(eps_dfs, ignore_index=True)

print(combined_eps_df)

C:\Users\mhunt\AppData\Local\Temp\ipykernel_7800\1151411026.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  eps_df = pd.read_html(table.get_attribute('outerHTML'))[0]
C:\Users\mhunt\AppData\Local\Temp\ipykernel_7800\1151411026.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  eps_df = pd.read_html(table.get_attribute('outerHTML'))[0]
C:\Users\mhunt\AppData\Local\Temp\ipykernel_7800\1151411026.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  eps_df = pd.read_html(table.get_attribute('outerHTML'))[0]
C:\Users\mhunt\AppData\Local\Temp\ipykernel_7800\1151411026.py:17: FutureWarning: Passing literal html to 're

            EPS FQ1 '20 FQ2 '20 FQ3 '20 FQ4 '20 FQ1 '21 FQ2 '21 FQ3 '21  \
0      Estimize    0.33    0.11    0.42    0.47    0.58    0.74    0.54   
1          Mean    0.34    0.13    0.39    0.46    0.55    0.70    0.56   
2          High    0.52    0.50    0.62    0.91    0.85    1.25    1.10   
3           Low    0.20    0.02    0.15    0.28    0.28    0.35    0.30   
4      Wall St.    0.31    0.07    0.37    0.36    0.48    0.61    0.44   
..          ...     ...     ...     ...     ...     ...     ...     ...   
297    Wall St.    1.24    1.37    1.12    1.01    1.42    1.51    1.19   
298        High    1.26    1.40    1.22    1.11    1.46    1.58    1.22   
299         Low    1.18    1.32    1.06    0.94    1.29    1.38    1.14   
300     Actuals    1.37    1.42    1.17    1.16    1.63    1.64    1.26   
301  YoY Growth     22%     14%     10%      5%     19%     15%      8%   

    FQ4 '21 FQ1 '22 FQ2 '22 FQ3 '22 FQ4 '22 Ticker  
0      0.22    0.47    0.17    0.26    0.23   

C:\Users\mhunt\AppData\Local\Temp\ipykernel_7800\1151411026.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  eps_df = pd.read_html(table.get_attribute('outerHTML'))[0]


In [17]:
# Sending the data to a CSV file
combined_eps_df.to_csv('quarterly_eps_data.csv', index=False)

# Pulling the EPS estimate analysis data

In [49]:
# Making a DataFrame for storing the URL data
url_df = pd.DataFrame(columns=['Ticker', 'Quarter', 'Year', 'URL'])

# Preparing the URLs for data pulling
for index, row in ticker_list.iterrows():
    ticker = row['ticker']
    base_url = 'https://www.estimize.com/pg/fq1-2020?metric_name=eps&chart=historical'

    # Iterating over quarters and years
    for year in range(2020, 2023):
        for quarter in ['q1', 'q2', 'q3', 'q4']:
            url = f"{base_url.replace('pg', ticker).replace('q1', quarter).replace('2020', str(year))}"
            new_row = {'Ticker': ticker, 'Quarter': quarter, 'Year': year, 'URL': url}
            url_df = pd.concat([url_df, pd.DataFrame([new_row])], ignore_index=True)

print(url_df)

    Ticker Quarter  Year                                                URL
0     AMZN      q1  2020  https://www.estimize.com/AMZN/fq1-2020?metric_...
1     AMZN      q2  2020  https://www.estimize.com/AMZN/fq2-2020?metric_...
2     AMZN      q3  2020  https://www.estimize.com/AMZN/fq3-2020?metric_...
3     AMZN      q4  2020  https://www.estimize.com/AMZN/fq4-2020?metric_...
4     AMZN      q1  2021  https://www.estimize.com/AMZN/fq1-2021?metric_...
..     ...     ...   ...                                                ...
355     PG      q4  2021  https://www.estimize.com/PG/fq4-2021?metric_na...
356     PG      q1  2022  https://www.estimize.com/PG/fq1-2022?metric_na...
357     PG      q2  2022  https://www.estimize.com/PG/fq2-2022?metric_na...
358     PG      q3  2022  https://www.estimize.com/PG/fq3-2022?metric_na...
359     PG      q4  2022  https://www.estimize.com/PG/fq4-2022?metric_na...

[360 rows x 4 columns]


In [65]:
#M Making an empty list in which to store the newly created dataframes
eps_analyst_dfs_list = []

# Iterating over the URL list and navigating to each URL in the webdriver
for _, row in url_df.iterrows():
    test_url = row['URL']
    driver.get(test_url)

    # Checking to see if the button at the bottom of the desired table that expands the table to all values exists (some tables only have a few entries and do not have this button)
    try:
        # Looking for the button for up to 10 seconds and assigning its link to the link_element variable if it is found
        link_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#estimates > div > a'))
        )
        # Clicking the link to load the full table
        link_element.click()
    except:
        # Printing that the button link was not found as well as the URL of the page to allow for followup investigation to confirm that no button was present
        # Critically, we do not end the for loop if the button is not found, as the lack of the button could simply mean that the table was already completely present on the page when our driver loaded the page
        print(f"Link not found or unable to click for URL: {test_url}")

    # Selecting the table
    try:
        table = driver.find_element(By.CSS_SELECTOR, '#estimates > table')
        
        # Extracting the table data
        table_html = table.get_attribute('outerHTML')

        # Converting the table data to a pandas DataFrame
        df = pd.read_html(table_html)[0]

        # Adding new columns and filling them with identifying values
        df['Ticker'] = row['Ticker']
        df['Quarter'] = row['Quarter']
        df['Year'] = row['Year']

        # Appending the individual DataFrame to the combined DataFrame list created earlier
        eps_analyst_dfs_list.append(df)
        
        print(f"Table found and converted to DataFrame for URL: {test_url}")
    except:
        print(f"Table NOT found for URL: {test_url}")

# Converting the DataFrame list into a DataFrame
combined_eps_analyst_df = pd.concat(eps_analyst_dfs_list, ignore_index=True)

C:\Users\mhunt\AppData\Local\Temp\ipykernel_7800\329258333.py:29: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table_html)[0]


Table found and converted to DataFrame for URL: https://www.estimize.com/AMZN/fq1-2020?metric_name=eps&chart=historical


C:\Users\mhunt\AppData\Local\Temp\ipykernel_7800\329258333.py:29: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table_html)[0]


Table found and converted to DataFrame for URL: https://www.estimize.com/AMZN/fq2-2020?metric_name=eps&chart=historical


C:\Users\mhunt\AppData\Local\Temp\ipykernel_7800\329258333.py:29: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table_html)[0]


...

C:\Users\mhunt\AppData\Local\Temp\ipykernel_7800\329258333.py:29: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table_html)[0]


Link not found or unable to click for URL: https://www.estimize.com/PG/fq3-2022?metric_name=eps&chart=historical
Table found and converted to DataFrame for URL: https://www.estimize.com/PG/fq3-2022?metric_name=eps&chart=historical


C:\Users\mhunt\AppData\Local\Temp\ipykernel_7800\329258333.py:29: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table_html)[0]


Link not found or unable to click for URL: https://www.estimize.com/PG/fq4-2022?metric_name=eps&chart=historical
Table found and converted to DataFrame for URL: https://www.estimize.com/PG/fq4-2022?metric_name=eps&chart=historical


C:\Users\mhunt\AppData\Local\Temp\ipykernel_7800\329258333.py:29: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table_html)[0]


In [66]:
combined_eps_analyst_df

,Chart,Unnamed: 1,Analyst,Rank,Points,Value,Confidence,Last Revised,Ticker,Quarter,Year
0,NaN,NaN,AMZN Reported EarningsAmazon.com Inc.,NaN,NaN,0.25,0.25,04/30/20,AMZN,q1,2020
1,NaN,NaN,Estimize Consensus385 estimates weighted,NaN,NaN,0.33,0.33,04/30/20,AMZN,q1,2020
2,NaN,NaN,Estimize Mean386 estimates averaged,NaN,NaN,0.34,0.34,04/30/20,AMZN,q1,2020
3,NaN,NaN,Wall Street Consensus,NaN,NaN,0.31,0.31,04/30/20,AMZN,q1,2020
4,Show EstimateHighlight analyst in all charts,+ Follow this analyst to receive alerts about ...,FlannerFlanner,1.0,25.0,0.25,5.0,04/20/20,AMZN,q1,2020
...,...,...,...,...,...,...,...,...,...,...,...
39219,Show EstimateHighlight analyst in all charts,+ Follow this analyst to receive alerts about ...,RWDKPDRWDKPD,20.0,-6.0,1.30,5.2,06/22/22,PG,q4,2022
39220,Show EstimateHighlight analyst in all charts,+ Follow this analyst to receive alerts about ...,ForniForni,21.0,-6.0,1.30,5.4,07/23/22,PG,q4,2022
39221,Show EstimateHighlight analyst in all charts,+ Follow this analyst to receive alerts about ...,DrognessDrogness,22.0,-10.0,1.34,2.3,01/21/22,PG,q4,2022
39222,Show EstimateHighlight analyst in all charts,+ Follow this analyst to receive alerts about ...,Analyst_9074721Analyst_9074721,23.0,-14.0,1.37,1.6,12/12/21,PG,q4,2022


In [67]:
combined_eps_analyst_df.to_csv('combined_eps_analysts.csv')

In [ ]:
#M Making an empty list in which to store the newly created dataframes
eps_analyst_dfs_list = []

# Iterating over the URL list and navigating to each URL in the webdriver
for _, row in url_df.iterrows():
    test_url = row['URL']
    driver.get(test_url)

    # Checking to see if the button at the bottom of the desired table that expands the table to all values exists (some tables only have a few entries and do not have this button)
    try:
        # Looking for the button for up to 10 seconds and assigning its link to the link_element variable if it is found
        link_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#estimates > div > a'))
        )
        # Clicking the link to load the full table
        link_element.click()
    except:
        # Printing that the button link was not found as well as the URL of the page to allow for followup investigation to confirm that no button was present
        # Critically, we do not end the for loop if the button is not found, as the lack of the button could simply mean that the table was already completely present on the page when our driver loaded the page
        print(f"Link not found or unable to click for URL: {test_url}")

    # Selecting the table
    try:
        table = driver.find_element(By.CSS_SELECTOR, '#estimates > table')
        
        # Extracting the table data
        table_html = table.get_attribute('outerHTML')

        # Converting the table data to a pandas DataFrame
        df = pd.read_html(table_html)[0]

        # Adding new columns and filling them with identifying values
        df['Ticker'] = row['Ticker']
        df['Quarter'] = row['Quarter']
        df['Year'] = row['Year']

        print(f"Table found and converted to DataFrame for URL: {test_url}")

        # Pulling the 
        
        # Appending the individual DataFrame to the combined DataFrame list created earlier
        eps_analyst_dfs_list.append(df)
           
    except:
        print(f"Table NOT found for URL: {test_url}")

# Converting the DataFrame list into a DataFrame
combined_eps_analyst_df = pd.concat(eps_analyst_dfs_list, ignore_index=True)

In [94]:
# Getting all usernames from desired ticker quarters

# Creating a set in which to house the unique usernames
unique_user_values = set()

# Iterating across the URL list as before
for _, row in url_df.iterrows():
    test_url = row['URL']
    driver.get(test_url)

    # Checking to see if the button at the bottom of the desired table that expands the table to all values exists (some tables only have a few entries and do not have this button)
    try:
        # Looking for the button for up to 10 seconds and assigning its link to the link_element variable if it is found
        link_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#estimates > div > a'))
        )
        # Clicking the link to load the full table
        link_element.click()
    except:
        # Printing that the button link was not found as well as the URL of the page to allow for followup investigation to confirm that no button was present
        # Critically, we do not end the for loop if the button is not found, as the lack of the button could simply mean that the table was already completely present on the page when our driver loaded the page
        print(f"Link not found or unable to click for URL: {test_url}")

    # Selecting the table
    try:
        table = driver.find_element(By.CSS_SELECTOR, '#estimates > table')
        
        # Extracting the table data
        table_html = table.get_attribute('outerHTML')

        # Parsing the data using BeautifulSoup
        soup = BeautifulSoup(table_html, 'html.parser')

        # Finding all elements in the soup object that are data-user elements (URL string to their Estimize profile page)
        user_elements = soup.find_all(attrs={'data-user': True})

        # Adding the elements to a list
        user_values = [element['data-user'] for element in user_elements]

        # Adding the list values to the previously created set, effectively eliminating duplicates
        unique_user_values.update(user_values)

        print(f"{len(unique_user_values)} Total Users found after URL: {test_url}")
    except:
        print(f"Table NOT found for URL: {test_url}")

# Converting the set to a list
final_unique_users = list(unique_user_values)
print(f"{len(final_unique_users)} Total Users found across all URLs")

401 Total Users found after URL: https://www.estimize.com/AMZN/fq1-2020?metric_name=eps&chart=historical
629 Total Users found after URL: https://www.estimize.com/AMZN/fq2-2020?metric_name=eps&chart=historical
746 Total Users found after URL: https://www.estimize.com/AMZN/fq3-2020?metric_name=eps&chart=historical
889 Total Users found after URL: https://www.estimize.com/AMZN/fq4-2020?metric_name=eps&chart=historical
971 Total Users found after URL: https://www.estimize.com/AMZN/fq1-2021?metric_name=eps&chart=historical
1042 Total Users found after URL: https://www.estimize.com/AMZN/fq2-2021?metric_name=eps&chart=historical
1106 Total Users found after URL: https://www.estimize.com/AMZN/fq3-2021?metric_name=eps&chart=historical
1225 Total Users found after URL: https://www.estimize.com/AMZN/fq4-2021?metric_name=eps&chart=historical
1294 Total Users found after URL: https://www.estimize.com/AMZN/fq1-2022?metric_name=eps&chart=historical
1344 Total Users found after URL: https://www.estim

In [99]:
# Converting the list of usernames to a CSV
output_csv_file = 'analyst_list.csv'

with open(output_csv_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['User'])
    writer.writerows([[user] for user in final_unique_users])

In [144]:
test_users = final_unique_users[5:11] 

# Creating list for the user tables
stocks_covered_dfs = []
pending_estimates_dfs = []
scored_estimates_dfs = []

first_attempt = True

# Iterating over URLs
for test_user in test_users:
    url = f"https://www.estimize.com/users/{test_user}"
    driver.get(url)

    # Checking for hidden entries in the Stocks Covered table
    if first_attempt:
        driver.execute_script("window.scrollBy(0, 500);")
        first_attempt = False
    else:
        while True:
            try:
                driver.execute_script("window.scrollBy(0, 500);")
                table = WebDriverWait(driver, 2).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '#profile-covered-stocks > div.linked-table'))
                )
                table_html = table.get_attribute('outerHTML')
                soup = BeautifulSoup(table_html, 'html.parser')
                hidden_entries = soup.find_all(class_='linked-row hidden')
    
                if not hidden_entries:
                    break
    
                pagination_link = driver.find_element(By.CSS_SELECTOR, '#profile-covered-stocks > div.pagination-footer')
                pagination_link.click()

            except Exception as e:
                print(f"\033[91mError while pulling Stocks Covered for User: {test_user}\033[0m")
                break
    
    sc_rows = soup.find_all('tr')
    sc_data = []
    for sc_row in sc_rows:
        sc_cols = sc_row.find_all('td')
        if sc_cols:
            sc_data.append({
                'User URL': test_user,
                'Analyst Username': test_user,
                'Ticker': cols[0].text,
                'Reports': cols[1].text,
                'Quarters': cols[2].text,
                'Points': cols[3].text,
                'Pts/Est': cols[4].text,
                'Error Rate': cols[5].text,
            })

    sc_df = pd.DataFrame(sc_data)

    stocks_covered_dfs.append(sc_df)

    print(f"Stocks Covered Scrape Complete for User: {test_user}")

    while True:
        try:
            driver.execute_script("window.scrollBy(0, 500);")
            table = WebDriverWait(driver, 0.5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#profile-pending-estimates > div.linked-table'))
            )
            table_html = table.get_attribute('outerHTML')
            soup = BeautifulSoup(table_html, 'html.parser')
            hidden_entries = soup.find_all(class_='linked-row hidden')

            if not hidden_entries:
                break

            pagination_link = driver.find_element(By.CSS_SELECTOR, '#profile-pending-estimates > div.pagination-footer')
            pagination_link.click()

        except Exception as e:
            print(f"\033[91mError while pulling Pending Estimates for User: {test_user}\033[0m")
            break
    
    pe_rows = soup.find_all('tr')
    pe_data = []
    for pe_row in pe_rows:
        pe_cols = pe_row.find_all('td')
        if pe_cols:
            data.append({
                'User URL': test_user,
                'Analyst Username': test_user,
                'Ticker': cols[0].text,
                'Quarter': cols[1].text,
                'Reports': cols[2].text,
                'Published': cols[3].text,
                'EPS': cols[4].text,
                'Revenue': cols[5].text,
            })

    pe_df = pd.DataFrame(pe_data)

    pending_estimates_dfs.append(pe_df)

    print(f"Pending Estimates Scrape Complete for User: {test_user}")

    while True:
        try:
            driver.execute_script("window.scrollBy(0, 500);")
            table = WebDriverWait(driver, 0.5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#profile-scored-estimates > div.linked-table'))
            )
            table_html = table.get_attribute('outerHTML')
            soup = BeautifulSoup(table_html, 'html.parser')
            hidden_entries = soup.find_all(class_='linked-row hidden')

            if not hidden_entries:
                break

            pagination_link = driver.find_element(By.CSS_SELECTOR, '#profile-scored-estimates > div.pagination-footer')
            pagination_link.click()

        except Exception as e:
            print(f"\033[91mError while pulling Scored Estimates for User: {test_user}\033[0m")
            break
    
    se_rows = soup.find_all('tr')
    se_data = []
    for se_row in se_rows:
        se_cols = se_row.find_all('td')
        if se_cols:
            data.append({
                'User URL': test_user,
                'Analyst Username': test_user,
                'Ticker': cols[0].text,
                'Quarter': cols[1].text,
                'Reported': cols[2].text,
                'Rank': cols[3].text,
                'EPS Points': cols[4].text,
                'Revenue Points': cols[5].text,
                'Total Points': cols[6].text
            })

    se_df = pd.DataFrame(se_data)

    scored_estimates_dfs.append(se_df)

    print(f"Scored Estimates Scrape Complete for User: {test_user}")

Stocks Covered Scrape Complete for User: analyst_3926000
Error while pulling Pending Estimates for User: analyst_3926000
Pending Estimates Scrape Complete for User: analyst_3926000
Scored Estimates Scrape Complete for User: analyst_3926000
Stocks Covered Scrape Complete for User: analyst_4411670
Error while pulling Pending Estimates for User: analyst_4411670
Pending Estimates Scrape Complete for User: analyst_4411670
Scored Estimates Scrape Complete for User: analyst_4411670
Stocks Covered Scrape Complete for User: analyst_9593416
Error while pulling Pending Estimates for User: analyst_9593416
Pending Estimates Scrape Complete for User: analyst_9593416
Scored Estimates Scrape Complete for User: analyst_9593416
Stocks Covered Scrape Complete for User: rsreina
Pending Estimates Scrape Complete for User: rsreina
Scored Estimates Scrape Complete for User: rsreina
Stocks Covered Scrape Complete for User: analyst_8891778
Error while pulling Pending Estimates for User: analyst_8891778
Pending

In [143]:
stocks_covered_dfs

[Empty DataFrame
 Columns: []
 Index: [],
 Empty DataFrame
 Columns: []
 Index: [],
 Empty DataFrame
 Columns: []
 Index: [],
 Empty DataFrame
 Columns: []
 Index: [],
 Empty DataFrame
 Columns: []
 Index: [],
 Empty DataFrame
 Columns: []
 Index: []]

In [178]:
test_users = final_unique_users[5:11] 

# Creating list for the user tables
general_user_data_dfs = []
stocks_covered_dfs = []
pending_estimates_dfs = []
scored_estimates_dfs = []

first_attempt = True

# Iterating over URLs
for test_user in test_users:
    url = f"https://www.estimize.com/users/{test_user}"
    driver.get(url)

    name = driver.find_element(By.CSS_SELECTOR, "#users_show > div.main-content-wrapper.is-closed > div.before-main-wrapper.content-header-show > div:nth-child(1) > div.profile-left-column > div > h1 > a")
    analyst_username = name.text.strip()

    roles = []
    role1 = driver.find_element(By.CSS_SELECTOR, "#users_show > div.main-content-wrapper.is-closed > div.before-main-wrapper.content-header-show > div:nth-child(1) > div.profile-left-column > div > ul > li:nth-child(1)")
    analyst_role1 = role1.text.strip()
    roles.append(role1)

    try:
        role2 = driver.find_element(By.CSS_SELECTOR, "#users_show > div.main-content-wrapper.is-closed > div.before-main-wrapper.content-header-show > div:nth-child(1) > div.profile-left-column > div > ul > li:nth-child(2)")
        analyst_role2 = role2.text.strip()
        roles.append(role2)

    except Exception as e:
        break   

    try:
        role3 = driver.find_element(By.CSS_SELECTOR, "#users_show > div.main-content-wrapper.is-closed > div.before-main-wrapper.content-header-show > div:nth-child(1) > div.profile-left-column > div > ul > li:nth-child(3)")
        analyst_role3 = role3.text.strip()
        roles.append(role3)

    except Exception as e:
        break 

    join = driver.find_element(By.CSS_SELECTOR, "#users_show > div.main-content-wrapper.is-closed > div.before-main-wrapper.content-header-show > div:nth-child(1) > div.profile-left-column > div > div.profile-bio-footer > div.profile-activity-stats")
    join_date = join.text.strip()

    confidence = driver.find_element(By.CSS_SELECTOR, "#confidence-wrap > div > div.value")
    analyst_confidence = confidence.text.strip()

    error = driver.find_element(By.CSS_SELECTOR, "#profile-tab-wrap > div.profile-tab.profile-tab-selectable.profile-tab-accuracy.is-selected > div:nth-child(1) > div.profile-stat-secondary")
    error_rate = error.text.strip()

    accuracy = driver.find_element(By.CSS_SELECTOR, "#profile-tab-wrap > div.profile-tab.profile-tab-selectable.profile-tab-accuracy.is-selected > div:nth-child(2) > div.profile-stat-secondary")
    accuracy_percentile = accuracy.text.strip()

    points = driver.find_element(By.CSS_SELECTOR, "#profile-tab-wrap > div.profile-tab.profile-tab-selectable.not-selected.profile-tab-points > div:nth-child(1) > div.profile-stat-secondary")
    points_total = points.text.strip()

    stocks = driver.find_element(By.CSS_SELECTOR, "#profile-tab-wrap > div.profile-tab.profile-tab-unselectable.profile-tab-last.not-selected.profile-tab-stocks > div:nth-child(1) > div.profile-stat-secondary")
    stocks_total = stocks.text.strip()

    pending = driver.find_element(By.CSS_SELECTOR, "#profile-tab-wrap > div.profile-tab.profile-tab-unselectable.profile-tab-last.not-selected.profile-tab-stocks > div:nth-child(2) > div.profile-stat-secondary")
    pending_estimates = pending.text.strip()
    
    # Create a DataFrame with the extracted value
    gud_df = pd.DataFrame({'Analyst Username': analyst_username, 
                           'URL Username': test_user,
                           'Roles': [roles],
                           'Join Date': join_date,
                           'Analyst Confidence Score': analyst_confidence,
                           'Error Rate': error_rate,
                           'Accuracy Percentile': accuracy_percentile,
                           'Points': points_total,
                           'Stocks': stocks_total,
                           'Pending': pending_estimates
                          })

    general_user_data_dfs.append(gud_df)
    
    # Checking for hidden entries in the Stocks Covered table
    if first_attempt:
        driver.execute_script("window.scrollBy(0, 500);")
        first_attempt = False
    else:
        while True:
            try:
                driver.execute_script("window.scrollBy(0, 500);")
                table = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '#profile-covered-stocks > div.linked-table'))
                )
                table_html = table.get_attribute('outerHTML')
                soup = BeautifulSoup(table_html, 'html.parser')
                hidden_entries = soup.find_all(class_='linked-row hidden')
    
                if not hidden_entries:
                    break
    
                pagination_link = driver.find_element(By.CSS_SELECTOR, '#profile-covered-stocks > div.pagination-footer')
                pagination_link.click()

            except Exception as e:
                print(f"\033[91mError while pulling Stocks Covered for User: {test_user}\033[0m")
                break

    lines = str(soup).splitlines()

    # Initialize an empty list to store the data
    sc_data = []
    
    # Extract data following the specified pattern
    for i in range(len(lines)):
        if "profile-tbl-ticker" in lines[i]:
            ticker = lines[i+1].strip()
            reports = lines[i + 6].strip()
            quarters = lines[i + 9].strip()
            points = lines[i + 12].strip()
            pts_per_estimate = lines[i + 15].strip()
            error_rate = lines[i + 18].strip()
            accuracy = lines[i + 21].strip()
    
            sc_data.append({
                'Analyst Username': analyst_username, 
                'URL Username': test_user,
                'Reports': reports,
                'Quarters': quarters,
                'Points': points,
                'Pts/Est': pts_per_estimate,
                'Error Rate': error_rate,
                'Accuracy': accuracy
            })
    
    # Create a DataFrame from the extracted data
    sc_df = pd.DataFrame(sc_data)

    stocks_covered_dfs.append(sc_df)

    print(f"Stocks Covered Scrape Complete for User: {test_user}")

Stocks Covered Scrape Complete for User: analyst_3926000
Stocks Covered Scrape Complete for User: analyst_4411670
Stocks Covered Scrape Complete for User: analyst_9593416


In [189]:
test_users = final_unique_users[5:11] 

# Creating list for the user tables
general_user_data_dicts = []
stocks_covered_dfs = []
pending_estimates_dfs = []
scored_estimates_dfs = []

first_attempt = True

# Iterating over URLs
for test_user in test_users:
    url = f"https://www.estimize.com/users/{test_user}"
    driver.get(url)

    name = driver.find_element(By.CSS_SELECTOR, "#users_show > div.main-content-wrapper.is-closed > div.before-main-wrapper.content-header-show > div:nth-child(1) > div.profile-left-column > div > h1 > a")
    analyst_username = name.text.strip()

    # Checking for hidden entries in the Stocks Covered table
    if first_attempt:
        driver.execute_script("window.scrollBy(0, 500);")
        first_attempt = False
    else:
        while True:
            try:
                driver.execute_script("window.scrollBy(0, 500);")
                table = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '#profile-covered-stocks > div.linked-table'))
                )
                table_html = table.get_attribute('outerHTML')
                soup = BeautifulSoup(table_html, 'html.parser')
                hidden_entries = soup.find_all(class_='linked-row hidden')
    
                if not hidden_entries:
                    break
    
                pagination_link = driver.find_element(By.CSS_SELECTOR, '#profile-covered-stocks > div.pagination-footer')
                pagination_link.click()

            except Exception as e:
                print(f"\033[91mError while pulling Stocks Covered for User: {test_user}\033[0m")
                break

    lines = str(soup).splitlines()

    # Initialize an empty list to store the data
    sc_data = []
    
    # Extract data following the specified pattern
    for i in range(len(lines)):
        if "profile-tbl-ticker" in lines[i]:
            ticker = lines[i+1].strip()
            reports = lines[i + 6].strip()
            quarters = lines[i + 9].strip()
            points = lines[i + 12].strip()
            pts_per_estimate = lines[i + 15].strip()
            error_rate = lines[i + 18].strip()
            accuracy = lines[i + 21].strip()
    
            sc_data.append({
                'Analyst Username': analyst_username, 
                'URL Username': test_user,
                'Reports': reports,
                'Quarters': quarters,
                'Points': points,
                'Pts/Est': pts_per_estimate,
                'Error Rate': error_rate,
                'Accuracy': accuracy
            })
    
    # Create a DataFrame from the extracted data
    sc_df = pd.DataFrame(sc_data)

    stocks_covered_dfs.append(sc_df)

    
    roles = []
    role1 = driver.find_element(By.CSS_SELECTOR, "#users_show > div.main-content-wrapper.is-closed > div.before-main-wrapper.content-header-show > div:nth-child(1) > div.profile-left-column > div > ul > li:nth-child(1)")
    analyst_role1 = role1.text.strip()
    roles.append(role1)

    try:
        role2 = driver.find_element(By.CSS_SELECTOR, "#users_show > div.main-content-wrapper.is-closed > div.before-main-wrapper.content-header-show > div:nth-child(1) > div.profile-left-column > div > ul > li:nth-child(2)")
        analyst_role2 = role2.text.strip()
        roles.append(role2)

    except Exception as e:
        break   

    try:
        role3 = driver.find_element(By.CSS_SELECTOR, "#users_show > div.main-content-wrapper.is-closed > div.before-main-wrapper.content-header-show > div:nth-child(1) > div.profile-left-column > div > ul > li:nth-child(3)")
        analyst_role3 = role3.text.strip()
        roles.append(role3)

    except Exception as e:
        break 

    join = driver.find_element(By.CSS_SELECTOR, "#users_show > div.main-content-wrapper.is-closed > div.before-main-wrapper.content-header-show > div:nth-child(1) > div.profile-left-column > div > div.profile-bio-footer > div.profile-activity-stats")
    join_date = join.text.strip()

    confidence = driver.find_element(By.CSS_SELECTOR, "#confidence-wrap > div > div.value")
    analyst_confidence = confidence.text.strip()

    error = driver.find_element(By.CSS_SELECTOR, "#profile-tab-wrap > div.profile-tab.profile-tab-selectable.profile-tab-accuracy.is-selected > div:nth-child(1) > div.profile-stat-secondary")
    error_rate = error.text.strip()

    accuracy = driver.find_element(By.CSS_SELECTOR, "#profile-tab-wrap > div.profile-tab.profile-tab-selectable.profile-tab-accuracy.is-selected > div:nth-child(2) > div.profile-stat-secondary")
    accuracy_percentile = accuracy.text.strip()

    points = driver.find_element(By.CSS_SELECTOR, "#profile-tab-wrap > div.profile-tab.profile-tab-selectable.not-selected.profile-tab-points > div:nth-child(1) > div.profile-stat-secondary")
    points_total = points.text.strip()

    stocks = driver.find_element(By.CSS_SELECTOR, "#profile-tab-wrap > div.profile-tab.profile-tab-unselectable.profile-tab-last.not-selected.profile-tab-stocks > div:nth-child(1) > div.profile-stat-secondary")
    stocks_total = stocks.text.strip()

    pending = driver.find_element(By.CSS_SELECTOR, "#profile-tab-wrap > div.profile-tab.profile-tab-unselectable.profile-tab-last.not-selected.profile-tab-stocks > div:nth-child(2) > div.profile-stat-secondary")
    pending_estimates = pending.text.strip()
    
    # Create a DataFrame with the extracted value
    gud_dict = {
                'Analyst Username': analyst_username, 
                'URL Username': test_user,
                'Roles': [roles],
                'Join Date': join_date,
                'Analyst Confidence Score': analyst_confidence,
                'Error Rate': error_rate,
                'Accuracy Percentile': accuracy_percentile,
                'Points': points_total,
                'Stocks': stocks_total,
                'Pending': pending_estimates
                }

    general_user_data_dicts.append(gud_dict)
    

    print(f"Stocks Covered Scrape Complete for User: {test_user}")

Stocks Covered Scrape Complete for User: analyst_3926000
Stocks Covered Scrape Complete for User: analyst_4411670
Stocks Covered Scrape Complete for User: analyst_9593416


In [220]:
test_users = final_unique_users[0:1] 

# Creating list for the user tables
general_user_data_dicts = []
stocks_covered_dfs = []
pending_estimates_dfs = []
scored_estimates_dfs = []

first_attempt = True

# Iterating over URLs
for test_user in test_users:
    url = f"https://www.estimize.com/users/{test_user}"
    driver.get(url)

    name = driver.find_element(By.CSS_SELECTOR, "#users_show > div.main-content-wrapper.is-closed > div.before-main-wrapper.content-header-show > div:nth-child(1) > div.profile-left-column > div > h1 > a")
    analyst_username = name.text.strip()

    join = driver.find_element(By.CSS_SELECTOR, "#users_show > div.main-content-wrapper.is-closed > div.before-main-wrapper.content-header-show > div:nth-child(1) > div.profile-left-column > div > div.profile-bio-footer > div.profile-activity-stats")
    join_date = join.text.strip()

    confidence = driver.find_element(By.CSS_SELECTOR, "#confidence-wrap > div > div.value")
    analyst_confidence = confidence.text.strip()

    error = driver.find_element(By.CSS_SELECTOR, "#profile-tab-wrap > div.profile-tab.profile-tab-selectable.profile-tab-accuracy.is-selected > div:nth-child(1) > div.profile-stat-secondary")
    error_rate = error.text.strip()

    accuracy = driver.find_element(By.CSS_SELECTOR, "#profile-tab-wrap > div.profile-tab.profile-tab-selectable.profile-tab-accuracy.is-selected > div:nth-child(2) > div.profile-stat-secondary")
    accuracy_percentile = accuracy.text.strip()

    points = driver.find_element(By.CSS_SELECTOR, "#profile-tab-wrap > div.profile-tab.profile-tab-selectable.not-selected.profile-tab-points > div:nth-child(1) > div.profile-stat-secondary")
    points_total = points.text.strip()

    stocks = driver.find_element(By.CSS_SELECTOR, "#profile-tab-wrap > div.profile-tab.profile-tab-unselectable.profile-tab-last.not-selected.profile-tab-stocks > div:nth-child(1) > div.profile-stat-secondary")
    stocks_total = stocks.text.strip()

    pending = driver.find_element(By.CSS_SELECTOR, "#profile-tab-wrap > div.profile-tab.profile-tab-unselectable.profile-tab-last.not-selected.profile-tab-stocks > div:nth-child(2) > div.profile-stat-secondary")
    pending_estimates = pending.text.strip()

    selectors = [
    "#users_show > div.main-content-wrapper.is-closed > div.before-main-wrapper.content-header-show > div:nth-child(1) > div.profile-left-column > div > ul > li:nth-child(1)",
    "#users_show > div.main-content-wrapper.is-closed > div.before-main-wrapper.content-header-show > div:nth-child(1) > div.profile-left-column > div > ul > li:nth-child(2)",
    "#users_show > div.main-content-wrapper.is-closed > div.before-main-wrapper.content-header-show > div:nth-child(1) > div.profile-left-column > div > ul > li:nth-child(3)"
    ]
    
    # Initialize an empty list to store the role values
    roles = []
    
    # Iterate through each selector and extract the role text
    for selector in selectors:
        try:
            role_element = driver.find_element(By.CSS_SELECTOR, selector)
            role_text = role_element.text
            roles.append(role_text)
        except Exception as e:
            pass
    # Create a DataFrame with the extracted value
    gud_dict = {
                'Analyst Username': analyst_username, 
                'URL Username': test_user,
                'Join Date': join_date,
                'Roles': roles,
                'Analyst Confidence Score': analyst_confidence,
                'Error Rate': error_rate,
                'Accuracy Percentile': accuracy_percentile,
                'Points': points_total,
                'Stocks': stocks_total,
                'Pending': pending_estimates
                }

    general_user_data_dicts.append(gud_dict)
    
    # Checking for hidden entries in the Stocks Covered table
    if first_attempt:
        driver.execute_script("window.scrollBy(0, 500);")
        first_attempt = False
    else:
        while True:
            try:
                driver.execute_script("window.scrollBy(0, 500);")
                table = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '#profile-covered-stocks > div.linked-table'))
                )
                table_html = table.get_attribute('outerHTML')
                soup = BeautifulSoup(table_html, 'html.parser')
                hidden_entries = soup.find_all(class_='linked-row hidden')
    
                if not hidden_entries:
                    break
    
                pagination_link = driver.find_element(By.CSS_SELECTOR, '#profile-covered-stocks > div.pagination-footer')
                pagination_link.click()

            except Exception as e:
                print(f"\033[91mError while pulling Stocks Covered for User: {test_user}\033[0m")
                break

    lines = str(soup).splitlines()
    print(soup)
    # Initialize an empty list to store the data
    sc_data = []
    
    # Extract data following the specified pattern
    for i in range(len(lines)):
        if "profile-tbl-ticker" in lines[i]:
            ticker = lines[i+1].strip()
            reports = lines[i + 6].strip()
            quarters = lines[i + 9].strip()
            points = lines[i + 12].strip()
            pts_per_estimate = lines[i + 15].strip()
            error_rate = lines[i + 18].strip()
            accuracy = lines[i + 21].strip()
    
            sc_data.append({
                'Analyst Username': analyst_username, 
                'URL Username': test_user,
                'Ticker': ticker,
                'Reports': reports,
                'Quarters': quarters,
                'Points': points,
                'Pts/Est': pts_per_estimate,
                'Error Rate': error_rate,
                'Accuracy': accuracy
            })
    
    # Create a DataFrame from the extracted data
    sc_df = pd.DataFrame(sc_data)

    stocks_covered_dfs.append(sc_df)

    print(f"Stocks Covered Scrape Complete for User: {test_user}")

    while True:
        try:
            driver.execute_script("window.scrollBy(0, 500);")
            table = WebDriverWait(driver, 0.5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#profile-pending-estimates > div.linked-table'))
            )
            table_html = table.get_attribute('outerHTML')
            soup = BeautifulSoup(table_html, 'html.parser')
            hidden_entries = soup.find_all(class_='linked-row hidden')

            if not hidden_entries:
                break

            pagination_link = driver.find_element(By.CSS_SELECTOR, '#profile-pending-estimates > div.pagination-footer')
            pagination_link.click()

        except Exception as e:
            print(f"\033[91mError while pulling Pending Estimates for User: {test_user}\033[0m")
            break

    lines = str(soup).splitlines()
    
    # Initialize an empty list to store the data
    pe_data = []
    
    # Extract data following the specified pattern
    for i in range(len(lines)):
        if "profile-tbl-ticker" in lines[i]:
            ticker = lines[i].strip()
            quarter = f"{lines[i + 3].strip()} {lines[i + 4].strip()}"
            reports = f"{lines[i + 8].strip()} {lines[i + 10].strip()}"
            published = lines[i + 13].strip()
            eps = lines[i + 16].strip()
            revenue = lines[i + 19].strip()
    
            pe_data.append({
                'Analyst Username': analyst_username, 
                'URL Username': test_user,
                'Ticker': ticker,
                'Quarter': quarter,
                'Reports': reports,                
                'Published': published,
                'EPS': eps,
                'Revenue': revenue
            })

    pe_df = pd.DataFrame(pe_data)

    pending_estimates_dfs.append(pe_df)

    print(f"Pending Estimates Scrape Complete for User: {test_user}")

    while True:
        try:
            driver.execute_script("window.scrollBy(0, 500);")
            table = WebDriverWait(driver, 0.5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#profile-scored-estimates > div.linked-table'))
            )
            table_html = table.get_attribute('outerHTML')
            soup = BeautifulSoup(table_html, 'html.parser')
            hidden_entries = soup.find_all(class_='linked-row hidden')

            if not hidden_entries:
                break

            pagination_link = driver.find_element(By.CSS_SELECTOR, '#profile-scored-estimates > div.pagination-footer')
            pagination_link.click()

        except Exception as e:
            print(f"\033[91mError while pulling Scored Estimates for User: {test_user}\033[0m")
            break
    

    lines = str(soup).splitlines()
    
    # Initialize an empty list to store the data
    se_data = []
    
    # Extract data following the specified pattern
    for i in range(len(lines)):
        if "profile-tbl-ticker" in lines[i]:
            ticker = lines[i].strip()
            quarter = f"{lines[i + 3].strip()} {lines[i + 4].strip()}"
            reported = lines[i + 7].strip()
            rank = lines[i + 10].strip()
            eps_points = lines[i + 13].strip()
            revenue_points = lines[i + 16].strip()
            total_points = lines[i + 19].strip()
    
            se_data.append({
                'Analyst Username': analyst_username, 
                'URL Username': test_user,
                'Ticker': ticker,
                'Quarter': quarter,
                'Reported': reported,                
                'Rank': rank,
                'EPS Points': eps_points,
                'Revenue Points': revenue_points,
                'Total Points': total_points
            })

    se_df = pd.DataFrame(se_data)

    scored_estimates_dfs.append(se_df)

    print(f"Scored Estimates Scrape Complete for User: {test_user}")

<div class="linked-table">
<div class="thead">
<div class="tr">
<div class="th unsortable">Ticker</div>
<div class="th unsortable">Quarter</div>
<div class="th unsortable">Reported</div>
<div class="th unsortable">Rank</div>
<div class="th unsortable">EPS Points</div>
<div class="th unsortable">Revenue Points</div>
<div class="th unsortable">Total Points</div>
</div>
</div>
<div class="tbody">
<a class="linked-row" href="/amd/fq3-2020/users/analyst_8891778"><div class="td" data-hasqtip="true" data-tooltip-id="amd" data-tooltip-type="instrument">
<strong class="profile-tbl-ticker">AMD</strong>
</div>
<div class="td">
Q3
2020
</div>
<div class="td profile-tbl-bordered">
Oct 27, 2020
</div>
<div class="td">
108 / 234
</div>
<div class="td">
21
</div>
<div class="td">
-3
</div>
<div class="td">
18
</div>
</a><a class="linked-row" href="/tsla/fq3-2020/users/analyst_8891778"><div class="td" data-hasqtip="true" data-tooltip-id="tsla" data-tooltip-type="instrument">
<strong class="profile-tbl-

In [219]:
combined_stocks_covered_df = pd.concat(stocks_covered_dfs)

combined_stocks_covered_df.to_csv('combined_stocks_covered_df.csv', index=False)